In [191]:
import os
from os import listdir
from os.path import isfile, join
from fnmatch import fnmatch
import shutil
import time
import zipfile
import textract, re
import patoolib

In [254]:
#Definitions
def forsort(a):
    anis = 0;
    for i in range(len(a)):
        if a[i].isnumeric():
            anis*=10
            anis+= int(a[i])
    return anis

def keyWords(l):
    l=l.replace('(', ' ')
    l=l.replace(')', ' ')
    l=l.replace('[', ' ')
    l=l.replace(']', ' ')
    l=l.replace('-', ' ')
    l=l.replace('+', ' ')
    l=l.replace('*', ' ')
    l=l.replace('/', ' ')
    l=l.replace('=', ' ')
    l=l.replace('\n', ' ')
    l=l.replace('^', ' ')
    l = l.split(' ')
    l = {x for x in l if (len(x)>1 or x.isnumeric())}
    return l

def similar(s, t):
    inter = s.intersection(t)
    if len(t) == 0 and len(s) == 0:
        return 0.9
    if len(t) == 0 or len(s) == 0:
        return 0
    
    return max(len(inter)/len(t), len(inter)/len(s))

def isOK(s):
    global express
    flag = False
    per = 0
    per1 = 0
    for i in range(n):
        per = similar(s, express[i])
        #print(per, end=' ')
        per1 = max(per1, per)
        if per >= 0.5:
            flag = True
        
    #print(round(per1, 1), end = ' ')
    if flag == True:
        print("OK")
    if flag == False:
        print("Reconsidering...")
    return flag

def getID(a):
    backa=a
    a = a.replace('-', ' ')
    a = a.replace('_', ' ')
    a = a.replace('.', ' ')
    a = a.split()
    for x in a:
        if len(x) > 7 and x.isnumeric():
            return x
    return "Undefined ID "+backa+"\nMake sure to include your Student ID in the name of the zip file."

def onlyFileName(a):
    anis = ""
    for j in range(len(a)-1, 0, -1):
        if a[j] == '/':
            break
        anis += a[j]
    anis = anis[::-1]
    return anis

def fileExt(a):
    anis = ""
    for j in range(len(a)-1, 0, -1):
        if a[j] == '.':
            break
        anis += a[j]
    anis = anis[::-1]
    return anis

In [255]:
express = [{} for i in range(100)] #Express[i]: set of key words for question i
path = 'Desktop/MATLAB/ANS/'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
onlyfiles = sorted(onlyfiles, key=forsort)
n = 0 #Number of questions
numOfChar = 0

for file in onlyfiles:
    n += 1
    express[n-1] = set()
    print(file)
    file1 = open(path+file, 'r') 
    print(path+file)
    Lines = file1.readlines() 
    for line in Lines:
        express[n-1]=express[n-1].union(keyWords(line))
        numOfChar+=len(line)
    print(express[n-1])
print(numOfChar)

g.m
Desktop/MATLAB/ANS/g.m
{'function', 'out', 'tan'}
multi.m
Desktop/MATLAB/ANS/multi.m
{'in2', 'function', 'in1,in2', 'out2', 'mod', '2', 'out1', 'out1,out2', ',1', 'in1', 'multi'}
f.m
Desktop/MATLAB/ANS/f.m
{'function', 'out', '1', 'x.', '2'}
fg.m
Desktop/MATLAB/ANS/fg.m
{'function', 'feval', 'fg', "'g',x"}
gf.m
Desktop/MATLAB/ANS/gf.m
{'function', 'feval', 'gf', "'f',x"}
func.m
Desktop/MATLAB/ANS/func.m
{'function', 'x.', '2', 'x,y', 'y.', 'output', 'func'}
Task2.m
Desktop/MATLAB/ANS/Task2.m
{'the', 'str2', 'str1', 'when', 'b;', 'disp', 'raised', "'Enter:", 'res', 'answer', 'is', 'power', "';", 'num2str', 'to', "'The", 'str3', 'input'}
Task4a.m
Desktop/MATLAB/ANS/Task4a.m
{'X,Y', 'equal', 'x;', '0', '2', 'meshgrid', 'x,y', '10:2', 'axis', 'func1', 'func', 'pi', 'pi;', 'X,Y,func1', '0:pi', 'contour'}
Task4b.m
Desktop/MATLAB/ANS/Task4b.m
{'sin', '1', 'X,Y,f,20', '2', 'x,y', 'X,Y,f', '3', 'meshgrid', 'pi;', '0:pi', 'contourf', 'X,Y', 'x;', '10:2', 'figure', 'surf', 'contour'}
task6.m


In [261]:
seat = [{} for i in range(100)] #Express[i]: set of key words for question i for student
path = 'Desktop/MATLAB/'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
onlyfiles = sorted(onlyfiles, key=forsort)

for student in onlyfiles:
    numOfCharS=0
    flagDoc=False
    print('\n\n')
    path = 'Desktop/MATLAB/'
    print('student:', getID(student))
    os.mkdir('Desktop/MATLAB/seat/')
    shutil.copyfile(path+student,path+'seat/'+student)
    
    if student[-1] == 'p':
        with zipfile.ZipFile(path+'seat/'+student, 'r') as zip_ref: #Extracting the zip file
            zip_ref.extractall('Desktop/MATLAB/seat/')
    if student[-1] == 'r':
        shutil.rmtree('Desktop/MATLAB/seat/')
        print("rar file ignored...")
        continue
    if student[:3] == 'RAR':
        print("> Submitting a rar file will not be tolerated.")
        
    root = 'Desktop/MATLAB/seat/'
    pattern = "*.*"
    ansuer=[]
    for path, subdirs, files in os.walk(root):
        for name in files:
            if fnmatch(name, pattern):
                newFile = str(os.path.join(path, name))
                if newFile[-1] != 'p' and newFile[-1] != 'f' and newFile[-1] != 'x':
                    ansuer+=[newFile]
                elif newFile[-1] == 'x':
                    print(newFile[-4:], "files are not accepted.")
                    
    ansuer = sorted(ansuer, key=forsort) #Matlab files
    #print(ansuer)
    nn = 0
    mark = 0
    for file in ansuer: #Finding characteristics of each answer
        nn += 1
        seat[nn-1] = set()
        file1 = open(file, 'r')
        print(onlyFileName(file), end= ' ')
        fe = fileExt(file)
        if fe != 'm' and fe != 'mat' and fe != 'ma' and fe != 'fig' and fe != 'png' and fe != 'jpg':
            print("File ingnored.")
            continue
        if fe == 'fig' or fe == 'png' or fe == 'jpg':
            print("You can include it only in your report.")
            continue
        Lines = file1.readlines() 
        for line in Lines:
            seat[nn-1]=seat[nn-1].union(keyWords(line))
            numOfCharS+=len(line)
            
        if isOK(seat[nn-1]): #Grading
            mark+=1
        #print(seat[nn-1])

    
    mark = min(90*(mark/n), 90)
    
    mark = max(mark, min(90*(numOfCharS*1.1/numOfChar), 90))
    
    numOfRep = 0
    #Grading the report
    root = 'Desktop/MATLAB/seat/'
    pattern = "*.pdf"
    report="0"
    for path, subdirs, files in os.walk(root):
        for name in files:
            if fnmatch(name, pattern):
                report=os.path.join(path, name)
                numOfRep+=1


    if numOfRep > 1:
        print("Submit only one report file.")
    if len(report) < 3:
        print("Report not found. Report file MUST be a pdf file.")
    else:
        print("Report:", end=' ')
        b = os.path.getsize(report)
        
        if b//1024 >= 500: #Bonus
            mark+=10
            print("Bonus Report")
        
        if b//1024 >= 300: #Bonus
            mark+=10
            print("Bonus Report")
            
        if b//1024 >= 100:
            mark+=10
            print("Full mark")
        else:
            b =(b//1024)/100
            mark+= (b*10)
            print(str(b*100)+'%', 'Too vague / insufficient screenshots')

    print("\nOverall mark:", min(mark, 100))
        
    
    shutil.rmtree('Desktop/MATLAB/seat/')




student: Undefined ID دریافریورـ سری دومzip.zip
Make sure to include your Student ID in the name of the zip file.
Report: Bonus Report
Bonus Report
Full mark

Overall mark: 30.0



student: Undefined ID Parsa Abed - HW2.zip
Make sure to include your Student ID in the name of the zip file.
xsin.m OK
func.m OK
multi.m Reconsidering...
g.m OK
f.m OK
task2.m OK
task2.txt File ingnored.
task4.m OK
task4.txt File ingnored.
task6.txt File ingnored.
task6.m Reconsidering...
task8.m Reconsidering...
task8.txt File ingnored.
task10.txt File ingnored.
task12.txt File ingnored.
task12.m OK
task10_1.m OK
task10_2.m OK
task10_3.m OK
task10_4.m OK
Report not found. Report file MUST be a pdf file.

Overall mark: 90



student: Undefined ID RARhw2-mahyarMirbabaei.zip
Make sure to include your Student ID in the name of the zip file.
> Submitting a rar file will not be tolerated.
گزارش کار.txt File ingnored.
hw2.m OK
hw2q6.m OK
hw2q4p1.m OK
hw2q4p2.m OK
Report not found. Report file MUST be a pdf file